In [58]:
import pandas as pd
import numpy as np
import os
import re
from IPython.display import display
os.listdir('../../txtPuro')

['SERASA-R2794720.txt',
 'SERASA-R2794721.txt',
 'SERASA-R2794722.txt',
 'SERASA-R2794723.txt',
 'SERASA-R2794724.txt',
 'SERASA-R798713.txt',
 '~$RASA-R2794721.txt']

In [387]:
pefin = ['SERASA-R114892.txt','SERASA-R798713.txt']
refin = ['SERASA-R2794720.txt']

In [388]:
text = ""
with open('../../txtPuro/'+refin[0]) as infile:
    for line in infile:
        text += line

In [407]:
def teste():
    return 

In [408]:
d = teste()

In [409]:
type(d)

NoneType

In [410]:
d is None

True

In [389]:
cpnj = text[text.find('CNPJ:'):].split('\n')[0]
cpnj = cpnj.replace('CNPJ:','').strip()

In [411]:
tc, vc, df = search_pendence_table("REFIN")
df

,DATA,MODALIDADE,AVAL,VALOR CONTRATO,ORIGEM,LOCAL
0,20/03/2010,OPER.CAMBIO,N,426.172,CCL 09/39,BANCO DA AMAZONI
1,17/03/2010,OPER.CAMBIO,N,477.000,CCL 09/38,BANCO DA AMAZONI
2,12/03/2010,OPER.CAMBIO,N,475.530,CCL 09/36,BANCO DA AMAZONI
3,17/02/2010,OPER.CAMBIO,N,609.076,CCL09037,BANCO DA AMAZONI
4,03/02/2010,OPER.CAMBIO,N,1.843.200,CCL 09/016,BANCO DA AMAZONI


In [412]:

            data_pendence = {'TOTAL DE OCORRENCIAS':[tc], 'VALOR TOTAL':[vc]}
            pendence_df = pd.DataFrame(data_pendence)

In [413]:
pendence_df

,TOTAL DE OCORRENCIAS,VALOR TOTAL
0,5,3.830.978


In [374]:
def search_pendence_table(name_type):
    vector_aux = text[text.find(name_type):].split('\n')
    vector_aux = np.array(vector_aux)

    table_vector = vector_aux[:np.where(vector_aux == '')[0][0]]
    
    #----------------------------------------------------------------
    total_occurrence = table_vector[1].split('=')[1].strip()
    total_value = table_vector[2].split('=')[1].strip()
    #----------------------------------------------------------------
    
    line_aux = table_vector[4:]
    line_aux = np.array(line_aux)
    line_aux = line_aux[line_aux!='']

    table_columns = line_aux[0].split('   ')
    table_columns = np.array(table_columns)
    table_columns= table_columns[table_columns!='']
    table_columns = [i.strip().replace(':','') for i in table_columns]
    
    line_aux = line_aux[1:]
    
    data = {i:[] for i in table_columns}


    for line in line_aux:
        line_1 = re.sub(' +', ' ', line)
        line_1 = line_1.split(' ')

        line_2 = np.array(line.split('  '))
        line_2 = line_2[line_2!='']

        data['DATA'].append(line_1[0])
        data['MODALIDADE'].append(' '.join(line_2[0].split(' ')[1:]))
        data['AVAL'].append(line_2[1].strip().split(' ')[0])
        value_c = line.split('R$')[1].strip().split(' ')[0]
        data['VALOR CONTRATO'].append(value_c)


        origem_vector = line.split(value_c)[1].strip()
        aux_origem = np.array(list(origem_vector))
        x = np.array([str.isnumeric(i) for i in aux_origem])
        i, = np.where(x == True)
        i = i.max()

        data['ORIGEM'].append(origem_vector[:i+1].strip())
        data['LOCAL'].append(origem_vector[i+1:].strip())
    
    return total_occurrence, total_value, pd.DataFrame(data)

In [6]:
def search_table(name:str,\
                table_columns:list = [],\
                split_line:int = 0,\
                split_space:str = '  ',\
                print_table:bool = False) -> (str, pd.DataFrame):
        
    vector_aux = text[text.find(name):].split('\n')
    vector_aux = np.array(vector_aux)

    table_vector = vector_aux[:np.where(vector_aux == '')[0][0]]

    table_name = table_vector[0].split('  ')[0].strip()

    if len(table_columns) == 0:
        table_columns = table_vector[1].split('  ')
        table_columns = np.array(table_columns)
        table_columns= table_columns[table_columns!='']
        table_columns = [i.strip().replace(':','') for i in table_columns]

    line_aux = table_vector[2+split_line:]
    line_aux = np.array(line_aux)
    line_aux = line_aux[line_aux!='']
    line_aux = [np.array(i.split(split_space)) for i in line_aux]
    line_aux = [i[i!=''] for i in line_aux]

    data = {i:[] for i in table_columns}
    for l in line_aux:
        for j in range(len(table_columns)):
            try:
                data[table_columns[j]].append(l[j])
            except:
                data[table_columns[j]].append(np.nan)
    df = pd.DataFrame(data)

    if print_table:
        print(table_name)
        display(df)

    return table_name, df

In [5]:
import json

In [6]:
_, df = search_table('RELACIONAMENTO COM O MERCADO')

In [7]:
type(json.loads(df.T.to_json()))

dict

In [103]:
df.columns

Index(['0-6 MESES', '6MES-1ANO', '1-3ANOS', '3-5ANOS', '5-10ANOS', '+10ANOS',
       'INAT.'],
      dtype='object')

In [106]:

re.sub(' +', ' ', 'RELACIONAMENTO COM -           FACTORINGS')

'RELACIONAMENTO COM - FACTORINGS'

In [6]:
search_table('RELACIONAMENTO COM -           FACTORINGS')

RELACIONAMENTO COM -


,0-6 MESES,6MES-1ANO,1-3ANOS,3-5ANOS,5-10ANOS,+10ANOS,INAT.
0,16,11,9,3,11,6,0


In [23]:
table_columns = ['MES_1', 'QTD_1', 'MES_2', 'QTD_2', 'CINCO_ULTIMAS_CONSULTAS', 'QTD']
name_table, df = search_table('REGISTRO DE CONSULTAS', table_columns = table_columns, split_line = 1)

In [28]:
df_1_aux = df[['MES_1', 'QTD_1']]
df_1_aux.columns = ['MES','QTD']
df_2_aux = df[['MES_2', 'QTD_2']]
df_2_aux.columns = ['MES','QTD']

df_1 = pd.concat([df_1_aux,df_2_aux])

In [82]:
df_2 = df[['CINCO_ULTIMAS_CONSULTAS', 'QTD']].iloc[:-1]

In [90]:
type(obj)

NameError: name 'obj' is not defined

In [83]:
def extract_date(line):
    aux_ = line.strip().split(' ')
    date = aux_[0]
    name = ' '.join(aux_[1:])
    return pd.Series({'DATA':pd.to_datetime(date, format = '%d/%m/%Y'), 
                       'CINCO_ULTIMAS_CONSULTAS':name.strip()})
    
df_2['DATA'] = '' 
df_2[['DATA','CINCO_ULTIMAS_CONSULTAS']] = df_2['CINCO_ULTIMAS_CONSULTAS'].apply(extract_date)

In [85]:
df_2.columns = ['EMPRESA', 'QTD','DATA']

In [86]:
df_2

,EMPRESA,QTD,DATA
0,D. C. SECCO & CIA LTDA,1,2015-04-14
1,OPINIAO S/A,1,2015-04-13
2,RED SA,1,2015-04-13
3,PLATINUM FOMENTO MERCANTIL LTDA,1,2015-04-10
4,G. FIN FOMENTO MERCANTIL LTDA,1,2015-04-09


In [27]:
df_1 

,MES_1,QTD_1,MES_2,QTD_2,CINCO_ULTIMAS_CONSULTAS,QTD
0,MAR/15,31,AGO/14,17,14/04/2015 D. C. SECCO & CIA LTDA,1
1,FEV/15,28,JUL/14,12,13/04/2015 OPINIAO S/A,1
2,JAN/15,21,JUN/14,23,13/04/2015 RED SA,1
3,DEZ/14,24,MAI/14,32,10/04/2015 PLATINUM FOMENTO MERCANTIL LTDA,1
4,NOV/14,17,ABR/14,22,09/04/2015 G. FIN FOMENTO MERCANTIL LTDA,1
5,OUT/14,22,MAR/14,22,NaN,NaN


In [8]:
table_columns= ['PONTUAL_QTD','PONTUAL_%',
                '8-15_QTD','8-15_%',
                '16-30_QTD','16-30_%',
                '31-60_QTD','31-60_%',
                '+60_QTD','+60_%',
                'A VISTA_QTD']
search_table('HISTORICO DE PAGAMENTOS (QTDE DE TITULOS)', table_columns= table_columns, split_line =1)

HISTORICO DE PAGAMENTOS (QTDE DE TITULOS)


,PONTUAL_QTD,PONTUAL_%,8-15_QTD,8-15_%,16-30_QTD,16-30_%,31-60_QTD,31-60_%,+60_QTD,+60_%,A VISTA_QTD
0,1.390 100,0,0,0,0,0,0,0,0,1,NaN


In [9]:
table_columns= ['MES/ANO','PONTUAL_QTD','PONTUAL_%',
                '8-15_QTD','8-15_%',
                '16-30_QTD','16-30_%',
                '31-60_QTD','31-60_%',
                '+60_QTD','+60_%',
                'PMA A VISTA TOTAL','PMA A VISTA TOTAL_1','PMA A VISTA TOTAL_2']
search_table('HISTORICO DE PAGAMENTOS NO MERCADO (VALORES EM R$)', table_columns = table_columns, split_line = 1, split_space = ' ')

HISTORICO DE PAGAMENTOS NO MERCADO (VALORES EM R$)


,MES/ANO,PONTUAL_QTD,PONTUAL_%,8-15_QTD,8-15_%,16-30_QTD,16-30_%,31-60_QTD,31-60_%,+60_QTD,+60_%,PMA A VISTA TOTAL,PMA A VISTA TOTAL_1,PMA A VISTA TOTAL_2
0,ABR/15,40894,100,0,0,0,0,0,0,0,0,0,0,40894
1,MAR/15,180130,100,0,0,0,0,0,0,0,0,0,0,180130
2,FEV/15,222403,100,0,0,0,0,0,0,0,0,0,0,222403
3,JAN/15,180404,100,0,0,0,0,0,0,0,0,0,0,180404
4,DEZ/14,204476,100,0,0,0,0,0,0,0,0,0,0,204476
5,NOV/14,116716,100,0,0,0,0,0,0,0,0,0,0,116716
6,OUT/14,215249,100,0,0,0,0,0,0,0,0,0,0,215249
7,SET/14,339074,100,0,0,0,0,0,0,0,0,0,0,339074
8,AGO/14,194750,100,0,0,0,0,0,0,0,0,0,0,194750
9,JUL/14,235018,100,0,0,0,0,0,0,0,0,0,0,235018


In [88]:
table_columns= ['MES/ANO','PONTUAL_QTD','PONTUAL_%',
                '8-15_QTD','8-15_%',
                '16-30_QTD','16-30_%',
                '31-60_QTD','31-60_%',
                '+60_QTD','+60_%',
                'PMA A VISTA TOTAL','PMA A VISTA TOTAL_1','PMA A VISTA TOTAL_2']
_,df = search_table('HISTORICO DE PAGAMENTOS - FACTORINGS (VALORES EM R$)', table_columns = table_columns, split_line = 1, split_space = ' ')
df

,MES/ANO,PONTUAL_QTD,PONTUAL_%,8-15_QTD,8-15_%,16-30_QTD,16-30_%,31-60_QTD,31-60_%,+60_QTD,+60_%,PMA A VISTA TOTAL,PMA A VISTA TOTAL_1,PMA A VISTA TOTAL_2
0,MES/ANO,VALOR,%,VALOR,%,VALOR,%,VALOR,%,VALOR,%,(*),MES,NaN
1,ABR/15,40894,100,0,0,0,0,0,0,0,0,0,0,40894
2,MAR/15,83769,100,0,0,0,0,0,0,0,0,0,0,83769
3,FEV/15,185547,100,0,0,0,0,0,0,0,0,0,0,185547
4,JAN/15,131819,100,0,0,0,0,0,0,0,0,0,0,131819
5,DEZ/14,124776,100,0,0,0,0,0,0,0,0,0,0,124776
6,NOV/14,83223,100,0,0,0,0,0,0,0,0,0,0,83223
7,OUT/14,118992,100,0,0,0,0,0,0,0,0,0,0,118992
8,SET/14,207241,100,0,0,0,0,0,0,0,0,0,0,207241
9,AGO/14,160587,100,0,0,0,0,0,0,0,0,0,0,160587


In [11]:
table_columns= ['PONTUAL_MES/ANO',
                '8-15_QTD','8-15_%',
                '16-30_QTD','16-30_%',
                '31-60_QTD','31-60_%',
                '+60_QTD','+60_%',
                'PMA A VISTA TOTAL','PMA A VISTA TOTAL_1','PMA A VISTA TOTAL_2','PMA A VISTA TOTAL_3','PMA A VISTA TOTAL_4']
search_table('HISTORICO DE PAGAMENTOS - VISAO CEDENTE (VALORES EM R$ MILHARES)', table_columns = table_columns, split_line = 1, split_space = ' ')

HISTORICO DE PAGAMENTOS - VISAO CEDENTE (VALORES EM R$ MILHARES)


,PONTUAL_MES/ANO,8-15_QTD,8-15_%,16-30_QTD,16-30_%,31-60_QTD,31-60_%,+60_QTD,+60_%,PMA A VISTA TOTAL,PMA A VISTA TOTAL_1,PMA A VISTA TOTAL_2,PMA A VISTA TOTAL_3,PMA A VISTA TOTAL_4
0,DEZ/14,1,52,1,48,0,0,0,0,0,0,4,0,2
1,SET/14,637,100,0,0,0,0,0,0,0,0,0,0,637
2,AGO/14,2197,100,0,0,0,0,0,0,0,0,0,0,2197
3,JUL/14,2260,100,0,0,0,0,0,0,0,0,0,0,2260
4,JUN/14,1329,100,0,0,0,0,0,0,0,0,0,0,1329
5,MAI/14,3156,100,0,0,0,0,0,0,0,0,0,0,3156
6,ABR/14,3369,100,0,0,0,0,0,0,0,0,0,0,3369
7,TOTAL,12949,1,12950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MEDIA,1850,100,0,0,0,0,0,0,0,0,0,1850,NaN
9,*,PMA,=,PRAZO,MEDIO,DE,ATRASO,(EM,DIAS),NaN,NaN,NaN,NaN,NaN


In [12]:
search_table('EVOLUCAO DE COMPROMISSOS NO MERCADO (VALORES EM R$)')

EVOLUCAO DE COMPROMISSOS NO MERCADO (VALORES EM R$)


,MES/ANO,A VENCER
0,ABR/15,277.561
1,MAR/15,276.190
2,FEV/15,274.168
3,JAN/15,330.590
4,DEZ/14,243.762
5,NOV/14,253.957
6,OUT/14,282.092
7,SET/14,394.910
8,AGO/14,537.262
9,JUL/14,353.104


In [13]:
search_table('EVOLUCAO DE COMPROMISSOS - FACTORINGS (VALORES EM R$)', split_line = 1, table_columns = ['MES/ANO', 'A VENCER'])

EVOLUCAO DE COMPROMISSOS - FACTORINGS (VALORES EM R$) SACADO


,MES/ANO,A VENCER
0,ABR/15,277.561
1,MAR/15,238.724
2,FEV/15,206.899
3,JAN/15,265.386
4,DEZ/14,202.595
5,NOV/14,161.063
6,OUT/14,224.798
7,SET/14,292.615
8,AGO/14,370.235
9,JUL/14,236.064


In [14]:
search_table('EVOLUCAO DE COMPROMISSOS - VISAO CEDENTE (VALORES EM R$)')

EVOLUCAO DE COMPROMISSOS - VISAO CEDENTE (VALORES EM R$)


,MES/ANO,VENCIDOS,A VENCER,TOTAL
0,ABR/15,640.383,3.810,644.193
1,MAR/15,637.528,6.665,644.193
2,FEV/15,245.681,398.512,644.193
3,JAN/15,0,644.194,644.194
4,DEZ/14,0,520.128,520.128
5,NOV/14,0,523.139,523.139
6,AGO/14,0,637.109,637.109
7,JUL/14,0,1867.164,1.867.164
8,JUN/14,0,2260.472,2.260.472
9,MAI/14,0,331.895,331.895


In [15]:
search_table('REFERENCIAIS DE NEGOCIOS NO MERCADO (VALORES EM R$)', split_space = '    ')

REFERENCIAIS DE NEGOCIOS NO MERCADO (VALORES EM R$)


,DATA,VALOR,MEDIA
0,ULTIMA COMPRA 10/04/2015,17.524,4.391
1,MAIOR FATURA 19/08/2014,81.215,14.832
2,MAIOR ACUMULO 07/03/2014,283.021,35.543


In [ ]:
name:str,
table_columns:list,
split_line:int,
split_space:str,
print_table:bool

In [16]:
search_table('REFERENCIAIS DE NEGOCIOS A PRAZO - FACTORINGS (VALORES EM R$)', split_space = '    ', split_line = 1, table_columns = ['DATA', 'VALOR', 'MEDIA'])

REFERENCIAIS DE NEGOCIOS A PRAZO - FACTORINGS (VALORES EM R$) SACADO


,DATA,VALOR,MEDIA
0,ULTIMA COMPRA 10/04/2015,17.524,6.320
1,MAIOR FATURA 19/08/2014,81.215,20.093
2,MAIOR ACUMULO 19/01/2015,163.233,44.186


In [11]:
_, df = search_table('OCORRENCIAS MAIS RECENTES (ATE 05)', table_columns = ['DATA','AVAL','VALOR CONTRATO','ORIGEM','LOCAL'])
df

,DATA,AVAL,VALOR CONTRATO,ORIGEM,LOCAL
0,20/03/2010 OPER.CAMBIO,N,R$,426.172 CCL 09/39,BANCO DA AMAZONI
1,17/03/2010 OPER.CAMBIO,N,R$,477.000 CCL 09/38,BANCO DA AMAZONI
2,12/03/2010 OPER.CAMBIO,N,R$,475.530 CCL 09/36,BANCO DA AMAZONI
3,17/02/2010 OPER.CAMBIO,N,R$,609.076 CCL09037,BANCO DA AMAZONI
4,03/02/2010 OPER.CAMBIO,N,R$ 1.843.200 CCL 09/016,BANCO DA AMAZONI,NaN


In [151]:
new_text = text[text.find('PENDENCIA:REFIN'):]

In [156]:
line_aux = line_aux[1:]

In [157]:
line_aux

array(['20/03/2010 OPER.CAMBIO   N  R$   426.172 CCL 09/39        BANCO DA AMAZONI',
       '17/03/2010 OPER.CAMBIO   N  R$   477.000 CCL 09/38        BANCO DA AMAZONI',
       '12/03/2010 OPER.CAMBIO   N  R$   475.530 CCL 09/36        BANCO DA AMAZONI',
       '17/02/2010 OPER.CAMBIO   N  R$   609.076 CCL09037         BANCO DA AMAZONI',
       '03/02/2010 OPER.CAMBIO   N  R$ 1.843.200 CCL 09/016       BANCO DA AMAZONI'],
      dtype='<U79')

In [158]:
table_columns

['DATA', 'MODALIDADE', 'AVAL', 'VALOR CONTRATO', 'ORIGEM', 'LOCAL']

In [161]:
pd.DataFrame(data)

,DATA,MODALIDADE,AVAL,VALOR CONTRATO,ORIGEM,LOCAL
0,20/03/2010,OPER.CAMBIO,N,426.172,CCL 09/39,BANCO DA AMAZONI
1,17/03/2010,OPER.CAMBIO,N,477.000,CCL 09/38,BANCO DA AMAZONI
2,12/03/2010,OPER.CAMBIO,N,475.530,CCL 09/36,BANCO DA AMAZONI
3,17/02/2010,OPER.CAMBIO,N,609.076,CCL09037,BANCO DA AMAZONI
4,03/02/2010,OPER.CAMBIO,N,1.843.200,CCL 09/016,BANCO DA AMAZONI
